In [1]:
'''
Plano de ação do caso de negócios
1. Pré-processamento dos dados
2. Criar o algortimo de ML
'''

'\nPlano de ação do caso de negócios\n1. Pré-processamento dos dados\n2. Criar o algortimo de ML\n'

In [2]:
# Bibliotecas
import numpy as np
from sklearn import preprocessing
import tensorflow as tf

In [3]:
# Datset
raw_csv_data = np.loadtxt("Audiobooks_data.csv", delimiter=",")
unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

In [4]:
# Balanceamento do Dataset
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
  if targets_all[i] == 0:
    zero_targets_counter += 1
    if zero_targets_counter > num_one_targets:
      indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis = 0)

In [5]:
# Padronização do Dataset
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

In [6]:
# "Embaralhamento" do Dataset
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

In [7]:
# Divisão do Dataset em Treinamento, Validação e Teste
samples_count = shuffled_inputs.shape[0]
train_sample_count = int(0.8 * samples_count)
validation_sample_count = int(0.1 * samples_count)
test_sample_count = samples_count - train_sample_count - validation_sample_count

train_inputs = shuffled_inputs[:train_sample_count]
train_targets = shuffled_targets[:train_sample_count]

validation_inputs = shuffled_inputs[train_sample_count:train_sample_count+validation_sample_count]
validation_targets = shuffled_targets[train_sample_count:train_sample_count+validation_sample_count]

test_inputs = shuffled_inputs[train_sample_count+validation_sample_count:]
test_targets = shuffled_targets[train_sample_count+validation_sample_count:]

print(f"Treinamento: {np.sum(train_targets)}, {train_sample_count}, {np.sum(train_targets) / train_sample_count}")
print(f"Validação: {np.sum(validation_targets)}, {validation_sample_count}, {np.sum(validation_targets) / validation_sample_count}")
print(f"Teste: {np.sum(test_targets)}, {test_sample_count}, {np.sum(test_targets) / test_sample_count}")

Treinamento: 1791.0, 3579, 0.5004191114836547
Validação: 223.0, 447, 0.4988814317673378
Teste: 223.0, 448, 0.49776785714285715


In [8]:
# Salvando o Dataset no Formato .npz
np.savez("Audiobooks_data_train", inputs=train_inputs, targets=train_targets)
np.savez("Audiobooks_data_validation", inputs=validation_inputs, targets=validation_targets)
np.savez("Audiobooks_data_test", inputs=test_inputs, targets=test_targets)

In [9]:
# Definição do Modelo
# Entradas: 10; Saídas: 2; Camadas Ocultas: 2; Comprimento das Camadas: 50
npz = np.load("Audiobooks_data_train.npz")
train_inputs = npz["inputs"].astype(np.float64)
train_targets = npz["targets"].astype(np.int64)

npz = np.load("Audiobooks_data_validation.npz")
validation_inputs, validation_targets = npz["inputs"].astype(np.float64), npz["targets"].astype(np.int64)

npz = np.load("Audiobooks_data_test.npz")
test_inputs, test_targets = npz["inputs"].astype(np.float64), npz["targets"].astype(np.int64)

In [10]:
# Definição do Modelo (continuação)
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                            tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                            tf.keras.layers.Dense(hidden_layer_size, activation="softmax")
                            ])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

batch_size = 100
max_epochs = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)   # objeto de parada precoce

model.fit(train_inputs,
          train_targets,
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks=[early_stopping],
          validation_data=[validation_inputs, validation_targets],
          verbose=2)

Epoch 1/100
36/36 - 1s - loss: 3.1710 - accuracy: 0.5152 - val_loss: 2.1355 - val_accuracy: 0.7405 - 692ms/epoch - 19ms/step
Epoch 2/100
36/36 - 0s - loss: 1.1251 - accuracy: 0.8011 - val_loss: 0.5139 - val_accuracy: 0.8501 - 103ms/epoch - 3ms/step
Epoch 3/100
36/36 - 0s - loss: 0.3935 - accuracy: 0.8726 - val_loss: 0.3738 - val_accuracy: 0.8635 - 105ms/epoch - 3ms/step
Epoch 4/100
36/36 - 0s - loss: 0.3343 - accuracy: 0.8793 - val_loss: 0.3456 - val_accuracy: 0.8658 - 97ms/epoch - 3ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3152 - accuracy: 0.8849 - val_loss: 0.3318 - val_accuracy: 0.8725 - 78ms/epoch - 2ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3041 - accuracy: 0.8877 - val_loss: 0.3182 - val_accuracy: 0.8747 - 82ms/epoch - 2ms/step
Epoch 7/100
36/36 - 0s - loss: 0.2936 - accuracy: 0.8938 - val_loss: 0.3082 - val_accuracy: 0.8770 - 64ms/epoch - 2ms/step
Epoch 8/100
36/36 - 0s - loss: 0.2858 - accuracy: 0.8944 - val_loss: 0.3029 - val_accuracy: 0.8770 - 76ms/epoch - 2ms/step
Epoch 9/100


In [11]:
# Testando o Modelo
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)
print(f"Test loss: {test_loss}. Test accuracy: {test_accuracy}")

14/14 [==============================] - 0s 2ms/step - loss: 0.2106 - accuracy: 0.9263
Test loss: 0.21061734855175018. Test accuracy: 0.9263392686843872
